In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
! mkdir ~/.kaggle

In [2]:
! cp kaggle.json ~/.kaggle/

In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download sarahh222/distracteddriversrevampeddataset

 98% 885M/906M [00:03<00:00, 256MB/s]
100% 906M/906M [00:03<00:00, 273MB/s]


In [ ]:
! unzip distracteddriversrevampeddataset.zip

In [6]:
! kaggle datasets download rajeevsharma993/distracted-driver-dataset

100% 3.99G/4.00G [00:28<00:00, 177MB/s]
100% 4.00G/4.00G [00:28<00:00, 152MB/s]


In [ ]:
! unzip distracted-driver-dataset.zip

In [8]:
class_names = ['Safe driving',
              'Texting - right',
              'Talking on the phone - right',
              'Texting - left',
              'Talking on the phone - left',
              'Operating the radio',
              'Drinking',
              'Reaching behind',
              'Hair and makeup',
              'Talking to passenger']

In [10]:
import tensorflow

In [11]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_data = image_dataset_from_directory('/content/imgs/train',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)

test_data = image_dataset_from_directory('/content/test-revamped/test',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)

val_data = image_dataset_from_directory('/content/validation-revamped/validation',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)
testing_data = image_dataset_from_directory('/content/train-revamped/train',
                                       image_size=(128,128),
                                       batch_size=100,
                                       seed=123)

Found 22424 files belonging to 10 classes.
Found 4098 files belonging to 10 classes.
Found 5223 files belonging to 10 classes.
Found 13103 files belonging to 10 classes.


In [12]:
train_data.class_names = class_names
test_data.class_names = class_names
val_data.class_names = class_names
testing_data.class_names = class_names

In [13]:
train_data.class_names

['Safe driving',
 'Texting - right',
 'Talking on the phone - right',
 'Texting - left',
 'Talking on the phone - left',
 'Operating the radio',
 'Drinking',
 'Reaching behind',
 'Hair and makeup',
 'Talking to passenger']

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import pickle
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Input, BatchNormalization

In [ ]:
model=Sequential()

model.add(Conv2D(16,(3,3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))


model.summary()

In [19]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [20]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)

history = model.fit(x = train_data,
          epochs=10,
          validation_data = val_data,
          callbacks=[es])

Epoch 1/10
225/225 [==============================] - 42s 179ms/step - loss: 5.7890 - accuracy: 0.1440 - val_loss: 1.7955 - val_accuracy: 0.3464
Epoch 2/10
225/225 [==============================] - 40s 174ms/step - loss: 1.8256 - accuracy: 0.3125 - val_loss: 1.3153 - val_accuracy: 0.4963
Epoch 3/10
225/225 [==============================] - 44s 193ms/step - loss: 1.5270 - accuracy: 0.4152 - val_loss: 1.0548 - val_accuracy: 0.6469
Epoch 4/10
225/225 [==============================] - 40s 175ms/step - loss: 1.3905 - accuracy: 0.4780 - val_loss: 0.8922 - val_accuracy: 0.7601
Epoch 5/10
225/225 [==============================] - 42s 183ms/step - loss: 1.2605 - accuracy: 0.5329 - val_loss: 0.7939 - val_accuracy: 0.7553
Epoch 6/10
225/225 [==============================] - 41s 178ms/step - loss: 1.1562 - accuracy: 0.5820 - val_loss: 0.6746 - val_accuracy: 0.8223
Epoch 7/10
225/225 [==============================] - 44s 194ms/step - loss: 1.0606 - accuracy: 0.6220 - val_loss: 0.5203 - val_ac

In [23]:
pickle.dump(model, open('custom_model.pkl','wb'))

In [26]:
pickled_model = pickle.load(open('/content/custom_model.pkl','rb'))

## Predicting an image

In [28]:
import tensorflow as tf
import numpy as np


In [51]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('/content/imgs/train/c0/img_100026.jpg', target_size=(128, 128))

#preprocess the image
my_image = img_to_array(my_image)

#make the prediction
prediction = saved_model.predict(np.array([my_image]))

1/1 [==============================] - 0s 25ms/step


In [48]:
res=np.argmax(prediction)
result=class_names[res]

In [50]:
print(result)

Safe driving


## Prediction Ends

In [ ]:
pickled_model3.evaluate(test_data)

41/41 [==============================] - 19s 453ms/step - loss: 0.0097 - accuracy: 0.9971


[0.009682747535407543, 0.9970717430114746]

In [ ]:
pickled_model3.evaluate(testing_data)

132/132 [==============================] - 60s 453ms/step - loss: 0.0029 - accuracy: 0.9993


[0.0029310830868780613, 0.9993131160736084]

In [58]:
import glob 
import os
import random
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16,mobilenet_v3
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models,optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.imagenet_utils import preprocess_input 
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import resnet50
import keras
import numpy as np

In [59]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32,(3,3),activation = 'relu',input_shape = (128,128,3)))
cnn_model.add(Conv2D(64,(3,3),activation = 'relu',padding = 'same'))
cnn_model.add(Conv2D(128,(3,3),activation = 'relu',padding = 'same'))

cnn_model.add(BatchNormalization())


cnn_model.add(MaxPooling2D((2,2)))
cnn_model.add(Conv2D(128,(3,3),activation = 'relu',padding='same'))
cnn_model.add(Conv2D(126,(3,3),activation = 'relu',padding='same'))
cnn_model.add(BatchNormalization())

cnn_model.add(MaxPooling2D((2,2)))

cnn_model.add(Conv2D(256,(3,3),activation='relu'))
cnn_model.add(BatchNormalization())

cnn_model.add(MaxPooling2D((2,2)))
cnn_model.add(Conv2D(256,(3,3),activation='relu'))
cnn_model.add(BatchNormalization())


cnn_model.add(Flatten())
cnn_model.add(Dense(512,activation = 'relu'))
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(256,activation = 'relu'))
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(256,activation = 'relu'))
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(128,activation = 'relu'))
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(10,activation = 'softmax',name = 'output'))

In [ ]:
cnn_model.summary()

In [61]:
cnn_model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [62]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)

history = cnn_model.fit(x = train_data,
          epochs=10,
          validation_data = val_data,
          callbacks=[es])

Epoch 1/10
225/225 [==============================] - 126s 542ms/step - loss: 0.3136 - accuracy: 0.9102 - val_loss: 0.0965 - val_accuracy: 0.9757
Epoch 2/10
225/225 [==============================] - 122s 538ms/step - loss: 0.0395 - accuracy: 0.9889 - val_loss: 0.1955 - val_accuracy: 0.9399
Epoch 3/10
225/225 [==============================] - 122s 538ms/step - loss: 0.0468 - accuracy: 0.9863 - val_loss: 0.0194 - val_accuracy: 0.9948
Epoch 4/10
225/225 [==============================] - 123s 539ms/step - loss: 0.0153 - accuracy: 0.9953 - val_loss: 0.0091 - val_accuracy: 0.9979
Epoch 5/10
225/225 [==============================] - 123s 540ms/step - loss: 0.0255 - accuracy: 0.9932 - val_loss: 0.1407 - val_accuracy: 0.9602
Epoch 6/10
225/225 [==============================] - 123s 543ms/step - loss: 0.0119 - accuracy: 0.9967 - val_loss: 0.0245 - val_accuracy: 0.9906
Epoch 7/10
225/225 [==============================] - 122s 539ms/step - loss: 0.0163 - accuracy: 0.9954 - val_loss: 0.0584 -

# SAVING THE CNN MODEL IN JSON FORMAT

saving the model architecture in json file

In [72]:
model_json = cnn_model.to_json()
with open('json_cnn_model.json','w') as file:
  file.write(model_json)

saving the weight

In [73]:
cnn_model.save_weights('weights.h5')

In [75]:
from tensorflow.keras.models import model_from_json
with open('json_cnn_model.json','r') as file:
  model_json = file.read()

model = model_from_json(model_json)

In [77]:
model.load_weights('weights.h5')

In [79]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [80]:
model.evaluate(test_data)
model.evaluate(testing_data)

132/132 [==============================] - 24s 175ms/step - loss: 0.0165 - accuracy: 0.9954


[0.016475588083267212, 0.9954208731651306]

## JSON FORMAT ENDS

# SAVING THE MODEL IN PICKLE FORMAT

In [67]:
import pickle
pickle.dump(cnn_model, open('pickle_cnn.pkl','wb'))

In [68]:
pickled_model = pickle.load(open('pickle_cnn.pkl','rb'))

### PICKLE FORMAT ENDS

# SAVING THE MODEL IN HDF5 FORMAT

In [37]:
cnn_model.save('cnn_model.h5')

In [69]:
cnn_model = pickled_model

In [70]:
cnn_model.evaluate(test_data)
cnn_model.evaluate(testing_data)

132/132 [==============================] - 24s 171ms/step - loss: 0.0165 - accuracy: 0.9954


[0.016475586220622063, 0.9954208731651306]

In [39]:
saved_model = keras.models.load_model('cnn_model.h5')

In [40]:
saved_model.evaluate(test_data)

41/41 [==============================] - 8s 159ms/step - loss: 0.0271 - accuracy: 0.9919


[0.027085039764642715, 0.9919472932815552]

In [41]:
saved_model.evaluate(val_data)

53/53 [==============================] - 10s 159ms/step - loss: 0.0240 - accuracy: 0.9931


[0.02399641089141369, 0.9931074380874634]

In [46]:
saved_model.evaluate(testing_data)

132/132 [==============================] - 30s 215ms/step - loss: 0.0188 - accuracy: 0.9937


[0.018770286813378334, 0.9937418699264526]

## HDF5 FORMAT ENDS

In [ ]:
from google.colab import files
files.download('/content/cnn_model.pkl')

In [ ]:
cnn_model_augmented = Sequential()
cnn_model_augmented.add(Conv2D(32,(3,3),activation = 'relu',name = 'Conv_input',input_shape = (128,128,3)))
cnn_model_augmented.add(Conv2D(32,(3,3),activation = 'relu',name = 'Conv_2',padding = 'same'))
cnn_model_augmented.add(Conv2D(32,(3,3),activation = 'relu',name = 'Conv_3',padding = 'same'))

cnn_model_augmented.add(BatchNormalization())


cnn_model_augmented.add(MaxPooling2D((2,2),name = 'max_1'))
cnn_model_augmented.add(Conv2D(64,(3,3),activation = 'relu',name = 'Conv_4',padding='same'))
cnn_model_augmented.add(Conv2D(64,(3,3),activation = 'relu',name = 'Conv_5',padding='same'))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(MaxPooling2D((2,2),name = 'max_2'))

cnn_model_augmented.add(Conv2D(128,(3,3),activation='relu'))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(MaxPooling2D((2,2)))
cnn_model_augmented.add(Conv2D(128,(3,3),activation='relu'))
cnn_model_augmented.add(BatchNormalization())


cnn_model_augmented.add(Flatten())
cnn_model_augmented.add(Dense(512,activation = 'relu',name = 'D1',))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(Dense(256,activation = 'relu',name = 'D2'))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(Dense(256,activation = 'relu',name = 'D3'))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(Dense(128,activation = 'relu' ,name ='D4'))
cnn_model_augmented.add(BatchNormalization())

cnn_model_augmented.add(Dense(10,activation = 'softmax',name = 'output'))

In [ ]:
cnn_model_augmented.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)

history = cnn_model_augmented.fit(x = train_data,
          epochs=10,
          validation_data = val_data,
          callbacks=[es])

Epoch 1/10
225/225 [==============================] - 65s 244ms/step - loss: 0.2870 - accuracy: 0.9175 - val_loss: 0.1635 - val_accuracy: 0.9540
Epoch 2/10
225/225 [==============================] - 53s 233ms/step - loss: 0.0406 - accuracy: 0.9888 - val_loss: 0.1130 - val_accuracy: 0.9644
Epoch 3/10
225/225 [==============================] - 53s 232ms/step - loss: 0.0601 - accuracy: 0.9824 - val_loss: 0.0622 - val_accuracy: 0.9770
Epoch 4/10
225/225 [==============================] - 53s 230ms/step - loss: 0.0103 - accuracy: 0.9973 - val_loss: 0.0437 - val_accuracy: 0.9879
Epoch 5/10
225/225 [==============================] - 54s 236ms/step - loss: 0.0144 - accuracy: 0.9960 - val_loss: 0.1489 - val_accuracy: 0.9636
Epoch 6/10
225/225 [==============================] - 56s 244ms/step - loss: 0.0135 - accuracy: 0.9960 - val_loss: 0.0132 - val_accuracy: 0.9966
Epoch 7/10
225/225 [==============================] - 53s 231ms/step - loss: 0.0305 - accuracy: 0.9911 - val_loss: 0.2842 - val_ac

In [71]:
localhost_save_option = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")

In [ ]:
pickle.dump(cnn_model, open('cnn_model2.pkl','wb'))

In [ ]:
pickled_model2 = pickle.load(open('/content/cnn_model2.pkl','rb'))

In [ ]:
pickled_model2.evaluate(test_data)

41/41 [==============================] - 9s 147ms/step - loss: 0.0549 - accuracy: 0.9868


[0.05493588000535965, 0.9868228435516357]

In [ ]:
pickled_model2.evaluate(testing_data)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('/content/img_327.jpg', target_size=(128, 128))

#preprocess the image
my_image = img_to_array(my_image)

#make the prediction
prediction = pickled_model2.predict(np.array([my_image]))

1/1 [==============================] - 0s 20ms/step


In [ ]:
prediction

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
image=tf.keras.preprocessing.image.load_img('/content/imgs/train/c0/img_100026.jpg')
image=tf.image.resize(image,(128,128))
input_array=tf.keras.preprocessing.image.img_to_array(image)
input_array=np.array([input_array])
pred=pickled_model2.predict(input_array)
res=np.argmax(pred)
result=class_names[res]
print(pred)

1/1 [==============================] - 1s 581ms/step
[[9.9999201e-01 1.0860556e-06 6.7983166e-08 2.8446177e-06 1.3793826e-07
  2.6637167e-08 9.4483035e-09 4.4899945e-08 4.5921539e-07 3.3515198e-06]]


In [ ]:
print(result)

Operating the radio
